# Phase 6 — Hyperparameter Tuning and Optimization

Tune classical ML models with RandomizedSearchCV and demonstrate Keras Tuner setup for CNN/LSTM.

In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from scipy.stats import randint, uniform

# Load balanced dataset
X = pd.read_csv("../data/train_balanced.csv")
y = pd.read_csv("../data/train_balanced_labels.csv", squeeze=True)

# ---------------- RF tuning ----------------
param_dist_rf = {
    'n_estimators': [200, 300, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10]
}
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
search_rf = RandomizedSearchCV(rf, param_dist_rf, n_iter=6, scoring='f1_weighted', cv=3, n_jobs=-1, verbose=2)
search_rf.fit(X, y)
print("Best RF params:", search_rf.best_params_)
joblib.dump(search_rf.best_estimator_, "../trained_models/final_rf.pkl")

# ---------------- XGB tuning ----------------
param_dist_xgb = {
    'n_estimators': [200, 300, 500],
    'max_depth': [6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1]
}
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_jobs=-1, random_state=42)
search_xgb = RandomizedSearchCV(xgb, param_dist_xgb, n_iter=6, scoring='f1_weighted', cv=3, n_jobs=-1, verbose=2)
search_xgb.fit(X, y)
print("Best XGB params:", search_xgb.best_params_)
joblib.dump(search_xgb.best_estimator_, "../trained_models/final_xgb.pkl")

# ---------------- Keras Tuner placeholder for CNN ----------------
print("For CNN/LSTM tuning use Keras Tuner (example code in comments).")
# Example Keras Tuner usage is left in the notebook for interactive tuning (resource heavy).
